# Description
This notebook looks at the different outcome probabilities for a roll to cast system such as Dangerous Magic by Stephen Dobson on the [Basic Fantasy Showcase](https://www.basicfantasy.org/showcase.cgi).  For each die/dice type, e.g. 2d6, you define the probabilities for each possible outcome of a die/dice roll and how the outcomes are partitioned.  For example, using a 2d6 system a possible parition scheme is 2, 3-4, 5-9, 10-11, and 12, which gives five possible outcomes.  These are defined below in the roll_probs and parititions dictionaries respectively.  There is also a bonuses list to see how the probabilities change if a bonus is added to the roll.  For example, in the 2d6 context, a +1 bonus would exclude the first partition above since a 2 can never be rolled.

The motivation for this analysis came from reading the Dangerous Magic supplement.  Since the outcome of each roll with a 1d20 follows a uniform distribution, a comparison with non-uniform distributions, e.g. 2d6 and 3d6, seemed worthwhile.

# Modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(legacy='1.25')

# Figures

In [ ]:
roll_probs = {
    '2d6':{2:2.78,3:5.56,4:8.33,5:11.11,6:13.89,7:16.67,8:13.89,9:11.11,10:8.33,11:5.56,12:2.78},
    '3d6':{3:0.46,4:1.39,5:2.78,6:4.63,7:6.94,8:9.72,9:11.57,10:12.50,11:12.50,12:11.57,13:9.72,14:6.94,15:4.63,16:2.78,17:1.39, 18:0.46},
    '1d20':{1:5.0,2:5.0,3:5.0,4:5.0,5:5.0,6:5.0,7:5.0,8:5.0,9:5.0,10:5.0,11:5.0,12:5.0,13:5.0,14:5.0,15:5.0,16:5.0,17:5.0,18:5.0,19:5.0,20:5.0},
}
partitions = {
    '2d6':[(None,2),(3,4),(5,9),(10,11),(12,None)],
    '3d6':[(None,4),(5,7),(8,13),(14,16),(17,None)],
    '1d20':[(None,1),(2,5),(6,15),(16,19),(20,None)],
}
bonuses = range(0,4)
for die,probs in roll_probs.items():
    roll_probs = {}
    for bonus in bonuses:
        for partition_idx, partition in enumerate(partitions[die]):
            partition_sum = 0.0
            a,b = partition
            if a is None:
                roll_probs_key = f'{b}'
                sum_range = range(1,b-bonus+1)
            elif b is None:
                roll_probs_key = f'{a}'
                sum_range = range(a-bonus,len(probs)+1+bonus)
            else:
                roll_probs_key = f'{a}-{b}'
                sum_range = range(a-bonus,b-bonus+1)
            for roll in sum_range:
                if roll in probs.keys():
                    partition_sum += probs[roll]
            if roll_probs_key not in roll_probs.keys():
                roll_probs[roll_probs_key] = [partition_sum]
            else:
                roll_probs[roll_probs_key].append(partition_sum)
    
    x = np.arange(len(bonuses))  # the label locations
    width = 0.15  # the width of the bars
    multiplier = -1
    
    fig, ax = plt.subplots(figsize=(12,4))
    fig.suptitle(die)
    for attribute, measurement in roll_probs.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, np.array(measurement), width, label=attribute)
        ax.bar_label(rects, padding=3)
        multiplier += 1
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xticks(x + width, [f'{die}+{bonus}' for bonus in bonuses])
    ax.legend(loc='upper left', ncols=5)
    ax.set_ylim(0, 100)
    ax.grid()
    
    plt.show()
    plt.close()